In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0018'
spoofed = True
MAX_BIAS = 50

if kitti_seq == '0028':
    start_idx = 1550
else:
    start_idx = 0

In [ ]:
# Load results
results_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'results', 'spoofed' if spoofed else 'nominal')
results_files = os.listdir(results_path)

In [ ]:
# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

In [ ]:
fname = results_files[0]
fname.split('.')[0][-4:]

In [ ]:
def plot_trajectories(gt_enu, graph_positions, spoofed_positions=None, detect_idx=None):
    ATTACK_START = 1000

    fgo_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], name='FGO')
    gt_traj = go.Scatter(x=gt_enu[:,0], y=gt_enu[:,1], name='Ground-truth', line=dict(color='black'))
    start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=True)
    plot_data = [fgo_traj, gt_traj, start]

    if spoofed_positions is not None:
        spoof_traj = go.Scatter(x=spoofed_positions[:,0], y=spoofed_positions[:,1], 
            name='Spoofed', line=dict(color='red', dash='dash')) 
        spoof_start = go.Scatter(x=[spoofed_positions[1000,0]], y=[spoofed_positions[1000,1]], 
            name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=True)
        plot_data += [spoof_traj, spoof_start]

    if detect_idx is not None:
        detect = go.Scatter(x=[graph_positions[detect_idx,0]], y=[graph_positions[detect_idx,1]], 
            name='Detection', mode='markers', hovertext=str(detect_idx), marker=dict(size=10, color='green'), showlegend=True)
        plot_data += [detect]
    
    fig = go.Figure(data=plot_data)
    fig.update_layout(width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
    #fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
    )
    fig.update_xaxes(autorange=True)
    fig.show()

In [ ]:
#threshold = 137.20835412917324
N_SHIFT = 10

for fname in results_files:
    if fname.split('.')[0][-4:] == '-50m':
        results = np.load(os.path.join(results_path, fname))
        graph_positions = results['positions']
        gt_enu = gt_enu[:graph_positions.shape[0]]
        if spoofed:
            spoofed_positions = results['spoofed']
        else: 
            spoofed_positions = None

        qs = results['qs']
        threshold = results['threshold']
        if any(qs > threshold):
            detect_idx = N_SHIFT * np.argmax(qs > threshold)
        else:
            detect_idx = None

        plot_trajectories(gt_enu, graph_positions, spoofed_positions, detect_idx)

In [ ]:
from plotly.subplots import make_subplots

def plot_errors(fig, gt_enu, graph_positions):
    traj_len = len(gt_enu)
    
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,0] - gt_enu[:,0], name='x error'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,1] - gt_enu[:,1], name='y error'), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,2] - gt_enu[:,2], name='z error'), row=3, col=1)

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

for fname in results_files:
    if fname.split('.')[0][-4:] == '_50m':
        results = np.load(os.path.join(results_path, fname))
        graph_positions = results['positions']
        gt_enu = gt_enu[:graph_positions.shape[0]]
        plot_errors(fig, gt_enu, graph_positions)

fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

Error values